In [1]:
import os
import glob
import pandas as pd
from pathlib import Path

In [4]:
home = Path.home()
project_files = home / "balzac_project" / "balzac_project" / "data"

os.chdir(project_files)

In [8]:
char_freq = pd.read_csv("character_frequencies.csv")
metadata = pd.read_csv("character_metadata.csv")

In [23]:
char_freq["Frequency"] = char_freq["Frequency"].astype(int) # normalize frequency data
char_freq["Appearance_Type"] = char_freq["Frequency"].lt(5).map({True: "minor", False: "major"}) # map appearances based on frequency count

grouped = char_freq.groupby(["Character", "Appearance_Type"])["Novel"].apply(list).reset_index() # create lists of novels per char
pivoted = grouped.pivot(
    index="Character",
    columns="Appearance_Type",
    values="Novel"
) # merge rows of appearance_type

pivoted = pivoted.reset_index() 
pivoted["major"] = pivoted["major"].apply(lambda x: x if isinstance(x, list) else [])
pivoted["minor"] = pivoted["minor"].apply(lambda x: x if isinstance(x, list) else [])
pivoted = pivoted.rename(
    columns={"major": "major_appearances", "minor": "minor_appearances"}
)

In [25]:
new_metadata = metadata.merge(pivoted, on="Character", how="left")

In [27]:
new_metadata = new_metadata.drop(columns=["Verified Novels"])
new_metadata = new_metadata.rename(columns={"gender":"Gender", "variant_names":"Name Variants", "major_appearances":"Major Appearances",  "minor_appearances":"Minor Appearances"}) 

In [29]:
new_metadata.to_csv("character_metadata.csv")